In [1]:
!pip install uszipcode

Load all the necessary libraries for this peoject

In [2]:
from uszipcode import SearchEngine
import pandas as pd

import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

In [3]:
df_data_1 = pd.read_csv('~/sat_performance.csv', header = 1, dtype = {'District Code':str})
df_data_2 = pd.read_csv('~/SchoolLocSearch.csv', dtype = {'Org Code':str, 'Zip':str})

In [4]:
df_data_2.head()

,Org Name,Org Code,Org Type,Function,Contact Name,Address 1,Address 2,Town,State,Zip,Phone,Fax,Grade
0,Abby Kelley Foster Charter Public (District),04450000,Charter District,Charter School Leader,Brian Haas,10 New Bond Street,NaN,Worcester,MA,01606,508-854-8400,508-854-8484,"K,01,02,03,04,05,06,07,08,09,10,11,12"
1,Abington,00010000,Public School District,Superintendent,Peter Schafer,1071 Washington St.,NaN,Abington,MA,02351,781-982-2150,781-982-2157,"PK,K,01,02,03,04,05,06,07,08,09,10,11,12"
2,Academy Of the Pacific Rim Charter Public (Dis...,04120000,Charter District,Charter School Leader,Spencer Blasdale,1 Westinghouse Plaza Bldg B,NaN,Hyde Park,MA,02136,617-361-0050,617-361-0045,"05,06,07,08,09,10,11,12"
3,Acton (non-op),00020000,Public School District,Superintendent,Peter Light,16 Charter Road,NaN,Acton,MA,01720,978-264-4700,978-264-3340,NaN
4,Acton-Boxborough,06000000,Public School District,Superintendent,Peter Light,15 Charter Rd,NaN,Acton,MA,01720,978-264-4700,978-264-3340,"PK,K,01,02,03,04,05,06,07,08,09,10,11,12"


Store SAT scores in a  new dataframe and remove the last row from the original data file as it contains the "Total" line. The SAT scores are for 2017-18 year.

In [5]:
sat_scores = df_data_1.drop('Writing', axis = 1)
sat_scores = sat_scores.drop(index = 300)
sat_scores.head()

,District Name,District Code,Tests Taken,Reading / Writing,Math
0,Abby Kelley Foster Charter Public (District),04450000,119,536.0,526.0
1,Abington,00010000,139,550.0,535.0
2,Academy Of the Pacific Rim Charter Public (Dis...,04120000,2,NaN,NaN
3,Acton-Boxborough,06000000,419,656.0,684.0
4,Adams-Cheshire,06030000,76,522.0,522.0


Store the location indormation of the schools in a new data frame. Remove unnecessary data.  

In [6]:
drop_columns = ['Address 2', 'Phone', 'Fax', 'Grade']
school_loc = df_data_2.drop(drop_columns, axis = 1)
school_loc.head()

,Org Name,Org Code,Org Type,Function,Contact Name,Address 1,Town,State,Zip
0,Abby Kelley Foster Charter Public (District),04450000,Charter District,Charter School Leader,Brian Haas,10 New Bond Street,Worcester,MA,01606
1,Abington,00010000,Public School District,Superintendent,Peter Schafer,1071 Washington St.,Abington,MA,02351
2,Academy Of the Pacific Rim Charter Public (Dis...,04120000,Charter District,Charter School Leader,Spencer Blasdale,1 Westinghouse Plaza Bldg B,Hyde Park,MA,02136
3,Acton (non-op),00020000,Public School District,Superintendent,Peter Light,16 Charter Road,Acton,MA,01720
4,Acton-Boxborough,06000000,Public School District,Superintendent,Peter Light,15 Charter Rd,Acton,MA,01720


Now, find 5 best schools by the SAT score. Define the best schools as those having WReading/Writing SAT score of more than 570 and Math score more than 570.  The best schools are those where students' total score is higher than 1100, but we want both writing skills and math skills to be high enough.

In [7]:
sat_scores.columns

Index(['District Name', 'District Code', 'Tests Taken', 'Reading / Writing',
       'Math'],
      dtype='object')

In [8]:
writing_score = 570
math_score= 570
sat_scores['Total Score'] = sat_scores['Reading / Writing'] +  sat_scores['Math']
sat_scores['Good School'] = 0
sat_scores.loc[(sat_scores['Reading / Writing'] > writing_score) & (sat_scores['Math'] > math_score) , 'Good School'] = 1

In [9]:
# Create a new dataframe from school_loc that contains only Org Code, Address 1, Town, State and Zip.
# Merge the new data frame with the sat_scores

school_inf_list = ['Org Code','Address 1', 'Town', 'State', 'Zip']
school_info = school_loc[school_inf_list]
school_info = school_info.rename(columns = {'Org Code': 'District Code'})

school_info = school_info.merge(right = sat_scores, on = 'District Code')

school_info.head()

,District Code,Address 1,Town,State,Zip,District Name,Tests Taken,Reading / Writing,Math,Total Score,Good School
0,04450000,10 New Bond Street,Worcester,MA,01606,Abby Kelley Foster Charter Public (District),119,536.0,526.0,1062.0,0
1,00010000,1071 Washington St.,Abington,MA,02351,Abington,139,550.0,535.0,1085.0,0
2,04120000,1 Westinghouse Plaza Bldg B,Hyde Park,MA,02136,Academy Of the Pacific Rim Charter Public (Dis...,2,NaN,NaN,NaN,0
3,06000000,15 Charter Rd,Acton,MA,01720,Acton-Boxborough,419,656.0,684.0,1340.0,1
4,06030000,191 Church St,Cheshire,MA,01225,Adams-Cheshire,76,522.0,522.0,1044.0,0


Sort the schools by the total SAT score and choose 5 best school. The best schools have to be identified as "Good School" as defined above.

In [10]:
school_info = school_info.sort_values(by = 'Total Score', ascending = False)
top5_schools = school_info.head()
top5_schools['Good School'].all() == 1

True

In [11]:
top5_schools.reset_index(drop = True, inplace = True)
top5_schools

,District Code,Address 1,Town,State,Zip,District Name,Tests Taken,Reading / Writing,Math,Total Score,Good School
0,04680000,85 Prescott Street,Worcester,MA,01605,Ma Academy for Math and Science,56,700.0,757.0,1457.0,1
1,06000000,15 Charter Rd,Acton,MA,01720,Acton-Boxborough,419,656.0,684.0,1340.0,1
2,01550000,146 Maple Street,Lexington,MA,02420,Lexington,599,645.0,674.0,1319.0,1
3,04300000,201 Forest Street,Marlborough,MA,01752,Advanced Math and Science Academy Charter (Dis...,156,631.0,676.0,1307.0,1
4,06400000,120 Meriam Rd,Concord,MA,01742,Concord-Carlisle,305,631.0,644.0,1275.0,1


Now we need to gather the information to make call to the FourSquare database. We will update the dataframe with new columns, namely the zip code's latitude and longitude bounds of 4 points north, soth, east and west. 

In [12]:
search = SearchEngine(simple_zipcode=True)

In [13]:
num_of_zips = top5_schools.shape[0]

In [14]:
top5_schools

,District Code,Address 1,Town,State,Zip,District Name,Tests Taken,Reading / Writing,Math,Total Score,Good School
0,04680000,85 Prescott Street,Worcester,MA,01605,Ma Academy for Math and Science,56,700.0,757.0,1457.0,1
1,06000000,15 Charter Rd,Acton,MA,01720,Acton-Boxborough,419,656.0,684.0,1340.0,1
2,01550000,146 Maple Street,Lexington,MA,02420,Lexington,599,645.0,674.0,1319.0,1
3,04300000,201 Forest Street,Marlborough,MA,01752,Advanced Math and Science Academy Charter (Dis...,156,631.0,676.0,1307.0,1
4,06400000,120 Meriam Rd,Concord,MA,01742,Concord-Carlisle,305,631.0,644.0,1275.0,1


In [15]:
coords_df = pd.DataFrame(columns = ['bounds_west', 'bounds_east' ,'bounds_north', 'bounds_south'])

for i in range(num_of_zips):
    zip_info = search.by_zipcode(top5_schools.loc[i,'Zip']).to_dict()
    
    zip_data = {'bounds_west': [zip_info['bounds_west']], \
               'bounds_east' : [zip_info['bounds_east']], \
               'bounds_north': [zip_info['bounds_north']], \
               'bounds_south': [zip_info['bounds_south']]}
    coords_df = coords_df.append(pd.DataFrame.from_dict(zip_data), ignore_index = True)
    

In [16]:
coords_df

,bounds_west,bounds_east,bounds_north,bounds_south
0,-71.838922,-71.755751,42.314981,42.264392
1,-71.498546,-71.384899,42.533992,42.436937
2,-71.255717,-71.177885,42.490033,42.424904
3,-71.625825,-71.475505,42.380632,42.310977
4,-71.430389,-71.289302,42.504550,42.412715


In [17]:
top5_schools = top5_schools.merge(coords_df, left_index = True, right_index = True)

In [18]:
top5_schools

,District Code,Address 1,Town,State,Zip,District Name,Tests Taken,Reading / Writing,Math,Total Score,Good School,bounds_west,bounds_east,bounds_north,bounds_south
0,04680000,85 Prescott Street,Worcester,MA,01605,Ma Academy for Math and Science,56,700.0,757.0,1457.0,1,-71.838922,-71.755751,42.314981,42.264392
1,06000000,15 Charter Rd,Acton,MA,01720,Acton-Boxborough,419,656.0,684.0,1340.0,1,-71.498546,-71.384899,42.533992,42.436937
2,01550000,146 Maple Street,Lexington,MA,02420,Lexington,599,645.0,674.0,1319.0,1,-71.255717,-71.177885,42.490033,42.424904
3,04300000,201 Forest Street,Marlborough,MA,01752,Advanced Math and Science Academy Charter (Dis...,156,631.0,676.0,1307.0,1,-71.625825,-71.475505,42.380632,42.310977
4,06400000,120 Meriam Rd,Concord,MA,01742,Concord-Carlisle,305,631.0,644.0,1275.0,1,-71.430389,-71.289302,42.504550,42.412715


### Prepare creds for Foursquare

In [19]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [20]:
CLIENT_ID = '....' #  Foursquare ID
CLIENT_SECRET = '....' # Foursquare Secret
VERSION = '20190625' # Foursquare API version




###### Define a function to parse the result from a call to FourSquare database

In [21]:
def getNearbyVenues(zp, so_coord, no_coord, ea_coord, we_coord):
    
    venues_list=[]
    for zpp, so, no, ea, we in zip(zp, so_coord, no_coord, ea_coord, we_coord):
        print(zpp)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&sw={},{}&ne={},{}'.format(CLIENT_ID, \
                                                                                                                  CLIENT_SECRET,\
                                                                                                                  VERSION, \
                                                                                                                  so,\
                                                                                                                  we,\
                                                                                                                  no,\
                                                                                                                  ea )
        
    
        # make the GET request
        results = requests.get(url).json()
        #print(results)
        #answer = input('enter')
        results = results["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            zpp, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip',  
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a data frame that holds all the venues in the neighborhood of each zip code

In [22]:
zp_venues = getNearbyVenues(zp = top5_schools['Zip'],
                             so_coord = top5_schools['bounds_south'],
                             no_coord = top5_schools['bounds_north'], 
                             ea_coord = top5_schools['bounds_east'], 
                             we_coord = top5_schools['bounds_west']
                                  )

01605
01720
02420
01752
01742


In [23]:
zp_venues.groupby('Zip').count()

,Venue,Venue Latitude,Venue Longitude,Venue Category
Zip,,,,
01605,30,30,30,30
01720,30,30,30,30
01742,30,30,30,30
01752,30,30,30,30
02420,30,30,30,30


In [24]:
# perform one hot encoding

# one hot encoding
zp_venues_onehot = pd.get_dummies(zp_venues[['Venue Category']], prefix="", prefix_sep="")
zp_venues_onehot.head()

,American Restaurant,Art Gallery,Art Museum,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Bistro,Bookstore,...,Soccer Field,Spa,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
zp_venues_onehot.shape

(150, 70)

In [26]:
zp_venues_onehot_zip = zp_venues_onehot.copy()

zp_venues_onehot_zip.insert(0, 'Zip', list(zp_venues['Zip'])) 


## Clustering zipcodes

The idea of clustering here is to understand how similar the zipcodes are. In other words, since there are only 5 zipcodes, we will perform clustering with 5 centroids and then exammine each cluster for the number of zipcodes.

In [28]:
# set number of clusters
kclusters = 5

# run k-means clustering
#k_means = KMeans(n_clusters=kclusters, random_state=0).fit(zp_venues_onehot)
k_means = KMeans(init="k-means++", n_clusters=kclusters, n_init=10).fit(zp_venues_onehot)

# check cluster labels generated for each row in the dataframe
k_means.labels_[0:10] 

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0])

In [29]:
#insert cluster labels into the dataframe with the zipcodes
zp_venues_onehot_zip.insert(1, 'Clusters', k_means.labels_) 

In [30]:
zp_venues_onehot_zip.head()

,Zip,Clusters,American Restaurant,Art Gallery,Art Museum,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,...,Soccer Field,Spa,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,01605,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01605,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01605,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,01605,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01605,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Examine the clusters

In [92]:
zzz = zp_venues_onehot_zip.groupby(['Zip', 'Clusters'])['Zip'].count()

In [93]:
zzz = zzz.to_frame()
zzz.head()
zzz.rename(columns = {'Zip':'Count'}, inplace = True)
zzz.reset_index(level = [0,1], inplace = True)

In [96]:
zzz = zzz.pivot(index = 'Zip', columns = 'Clusters', values = "Count")

In [103]:
zzz = zzz.fillna(0)
zzz


Clusters,0,1,2,3,4
Zip,,,,,
01605,26.0,1.0,0.0,1.0,2.0
01720,25.0,1.0,1.0,2.0,1.0
01742,26.0,2.0,1.0,1.0,0.0
01752,22.0,2.0,2.0,3.0,1.0
02420,27.0,1.0,0.0,1.0,1.0


We can clearly see that each cluster contain more or less same number of zipcodes and hence the neighboorgoods look the same.
